# Gaia DR3 detectability of unresolved binary systems

updated 23 Apr. 2024

In this notebook, we discuss how to estimate the detectability of unresolved binary systems based on RUWE. 

We also present a forward model to simulate a RUWE value for well-behaved single sources or binary systems, based on the astrimetric goodness-of-fit. We present a sky-varying map of maximum RUWE values compatible with single sources, where sources with larger RUWE can be considered potential binary systems.

This work is currently based on precomputed maps at a resolution of HEALPix level 5. However, the procedure described here, further developed in [Castro-Ginard et al. (2024)](https://arxiv.org/abs/2404.14127), is applicable to any resolution (Gaia scanning law details from GOST are required).

### Sky-varying RUWE threshold

One of the main results in [Castro-Ginard et al. (2024)](https://arxiv.org/abs/2404.14127) is Fig. 3, which shows a sky-varying RUWE thresdhold above which soucers show potential signs of binarity. 

To reproduce the plot, and access these RUWE values, we need to import the relevant module from the GaiaUnlimited package and create the selection function object. 

In [ ]:
from gaiaunlimited.selectionfunctions import binaries

binariesSF = binaries.BinarySystemsSelectionFunction()

This object provides a pre-computed table at HEALPix level 5, which includes for each pixel the RUWE thresholds together with the Gaia scanning law details needed to compute them and the probability to select sources with a RUWE higher than the threshold (estimated as in [Castro-Ginard et al., 2023](https://ui.adsabs.harvard.edu/abs/2023A%26A...677A..37C/abstract)).

These values are accessible through their sky coordinates, using an astropy SkyCoord object. We generate the coordinates of the centres of each HEALPix at level 5 to generate an all-sky map.

In [ ]:
from gaiaunlimited import fetch_utils,utils

coords_of_centres = utils.get_healpix_centres(5)
coords_of_centres

Then, we query the values for the RUWE threshold with:

In [ ]:
ruwe_threshold = binariesSF.query_RUWE(coords_of_centres,crowding = True)

print("RUWE threshold per pixel:")
print(ruwe_threshold)

The argument "crowding = True" gives us the contribution to a increased RUWE from sources other than binarity, gauged directly from Gaia data. Setting "crowding = False" will provide a RUWE threshold where values above are only increased by binarity. The default and recommended setting for "crowding" is True.

To show a sky map of these values we use the healpy python package.

In [ ]:
import healpy as hp

plt.figure()
hp.mollview(ruwe_threshold)
plt.show()

The Gaia scanning law details to estimate the above values are also accessible through the selection function object we created. This will provide the observation times, scanning angles or the along-scan parallax factors for each CCD observation of a certain pixel (usually 9 CCD observations per source transit). The number of observations can be derived from the previous. These data are collected through the Gaia Observation Forecast Tool ([GOST](https://gaia.esac.esa.int/gost/)).

In [ ]:
observation_times, scanning_angles, AL_parallax_factors = binariesSF.query_ScanningLaw(coords_of_centres)

print("Number of CCD observations per pixel:")
n_observations = np.hstack([len(observation_times[i]) for i in range(hp.order2npix(5))])
print(n_observations)

plt.figure()
hp.mollview(n_observations)
plt.show()

### Selection function of potential binary systems based on RUWE

Based on this sky-varyin RUWE threshold we can estimate the probability to select sources with RUWE higher than the threshold, which can be seen as potential binary systems (note that there are other contributions that will lead to an increased RUWE which are not taken into account).

These probabilities are computed using the subsample selection function (link to notebook), and are dependend only on the sky coordinates. These probabilities are a multiplicative factor to the Gaia catalogue selection function(link to notebook), described in [Cantat-Gaudin et al. (2023)](https://ui.adsabs.harvard.edu/abs/2023A%26A...669A..55C/abstract), and which depends on the sky coordinates and magnitude.

In [ ]:
probability,variance = binariesSF.query(coords_of_centres,return_variance = True)

plt.figure()
hp.mollview(probability)
plt.show()

plt.figure()
hp.mollview(variance)
plt.show()

### Forward model to estimate the RUWE for a simulated source

In [ ]:
from gaiaunlimited.utils import SimulateGaiaSource

In [ ]:
ra = 
dec = 

source = SimulateGaiaSource(ra, dec, period = 0, eccentricity = 0, initial_phase = 0)

In [ ]:
Gmag = 
parallax = 
semimajor_axis = 
mass_ratio = 
luminosity_ratio = 
phi = 
theta = 
omega = 

al_position, al_error = source.observe(Gmag, parallax,\
                                       semimajor_axis,\
                                       mass_ratio, luminosity_ratio,\
                                       phi, theta, omega,)

In [ ]:
ruwe = source.unit_weight_error(al_position, al_error)

In [ ]:
Gmag = 
parallax = 
semimajor_axis = 
mass_ratio = 
luminosity_ratio = 
phi = 
theta = 
omega = 

al_positions, al_errors = source.observe(Gmag, parallax,\
                                       semimajor_axis,\
                                       mass_ratio, luminosity_ratio,\
                                       phi, theta, omega,)

In [ ]:
ruwes = source.unit_weight_error(al_positions, al_errors)

In [ ]:
plt.figure()
plt.hist(ruwes,bins = np.logspace(np.log10(0.5),np.log10(100),100))
plt.show()